Lectura de la señal, cuantificación y codificación PCM (codPCM)

In [1]:
import numpy as np
import struct
import soundfile as sf

# audio, fs = sf.read("../data/clarinete.wav")
# plt.plot(audio)


def codPCM(x, R, fi):
    """
    Codifica la señal x (numpy array float64 en [-1,1]) con cuantización uniforme de R bits
    y genera un fichero binario (fi) que contiene:
       - R (1 byte)
       - N (4 bytes, uint32)
       - k'[n] (N valores enteros sin signo)
         (donde k'[n] = k[n] + 2^(R-1) )
    """
    # Verificaciones básicas
    if R < 1 or R > 15:
        raise ValueError("R debe estar en [1..15].")
    if not isinstance(x, np.ndarray):
        raise ValueError("x debe ser un numpy array.")

    # 1) Número de muestras
    N = x.size

    # 2) Paso de cuantización
    Delta = 2.0 ** (1 - R)  # = 2^(1-R)

    # 3) Cuantización uniforme
    #   k = floor(x / Delta)
    k_index = np.floor(x / Delta)
    #   Aseguramos que sea int (aunque floor ya devuelve float con parte decimal 0)
    k_index = k_index.astype(np.int32)  # k en [-2^(R-1), 2^(R-1)-1]

    # 4) Convertimos a k' = k + 2^(R-1)  (sin signo)
    offset = 2 ** (R - 1)
    kprime = k_index + offset  # en [0, 2^R - 1]

    # En el peor caso R=15 => 2^15-1 = 32767 => cabe en uint16
    # Convertimos a uint16 para guardar
    kprime_uint16 = kprime.astype(np.uint16)

    with open(fi, "wb") as f:
        # Escribimos R en 1 byte
        f.write(struct.pack("B", R))

        # Escribimos N en 4 bytes (uint32, little-endian)
        f.write(struct.pack("<I", N))

        # Escribimos k'[n] (N valores) en formato "uint16" (2 bytes c/u, little-endian)
        # Podemos usar tobytes() (NumPy >= 1.9) o tofile():
        f.write(kprime_uint16.tobytes())

    print(f"[codPCM] Fichero '{fi}' escrito con {N} muestras y R={R} bits.")

Decodificador PCM (decPCM)


In [2]:
def decPCM(fi):
    """
    Lee el fichero binario fi (generado por codPCM) y reconstruye la señal y (np.float64).
    Retorna (y, R).
    """
    with open(fi, "rb") as f:
        # 1) Leer R (1 byte)
        R_data = f.read(1)
        R = struct.unpack("B", R_data)[0]

        # 2) Leer N (4 bytes, uint32)
        N_data = f.read(4)
        N = struct.unpack("<I", N_data)[0]

        # 3) Leer N valores uint16
        #   Cada valor = k'[n]
        kprime_data = f.read(2 * N)  # 2 bytes por muestra
        kprime_arr = np.frombuffer(kprime_data, dtype=np.uint16)

    # 4) k[n] = k'[n] - 2^(R-1)
    offset = 2 ** (R - 1)
    k_arr = kprime_arr.astype(np.int32) - offset

    # 5) y[n] = k[n] * 2^(1-R)
    Delta = 2.0 ** (1 - R)
    y = k_arr.astype(np.float64) * Delta

    print(f"[decPCM] Fichero '{fi}' leído. R={R}, N={N}.")
    return y, R

Ejemplo de uso completo

In [3]:
import soundfile as sf
import sounddevice as sd


def ejemplo_PCM():
    # 1) Leer vt1.wav
    x, fs = sf.read("../data/vt1.wav")  # x ~ señal en [-1,1]
    print(f"Leídos {len(x)} samples, fs={fs} Hz")

    # 2) Elegir R
    R = 13
    # 3) Codificar
    codPCM(x, R, "vt1_pcm.bin")

    # 4) Decodificar
    y, R_leido = decPCM("vt1_pcm.bin")

    # 5) Calcular error RMS o MSE
    mse = np.mean((x - y) ** 2)
    print(f"Error cuadrático medio (MSE) = {mse:.6e}")

    # 6) Reproducir original
    print("Reproduciendo señal original...")
    sd.play(x, fs)
    sd.wait()

    # 7) Reproducir señal decodificada
    print("Reproduciendo señal decodificada...")
    sd.play(y, fs)
    sd.wait()

    # Observa si hay diferencia audible
    print("Fin del ejemplo.")


ejemplo_PCM()

Leídos 32613 samples, fs=8000 Hz
[codPCM] Fichero 'vt1_pcm.bin' escrito con 32613 muestras y R=13 bits.
[decPCM] Fichero 'vt1_pcm.bin' leído. R=13, N=32613.
Error cuadrático medio (MSE) = 1.623271e-08
Reproduciendo señal original...
Reproduciendo señal decodificada...
Fin del ejemplo.
